In [33]:
%load_ext autoreload
%autoreload 2

from mountaintools import client as mt
import os
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
mt.configRemoteReadonly(collection='spikeforest', share_id='spikeforest.spikeforest2')

MOUNTAIN CONFIG: remote database spikeforest (readonly); remote kb-share 69432e9201d0 (readonly)


In [6]:
output_ids=['mearec_neuronexus', 'visapy_mea', 'magland_synth', 'paired', 'mearec_tetrode', 'manual_tetrode', 'bionet']

In [7]:
result_objects=[
    mt.loadObject(
        key=dict(
            name='spikeforest_results'
        ),
        subkey=output_id
    )
    for output_id in output_ids
]
studies=[study for X in result_objects for study in X['studies']]
recordings=[recording for X in result_objects for recording in X['recordings']]
sorting_results=[sorting_result for X in result_objects for sorting_result in X['sorting_results']]

Downloaded file (878.6 KiB) in 2.1449368000030518 sec.
Downloaded file (230.0 KiB) in 1.2338745594024658 sec.
Downloaded file (116.1 KiB) in 0.9306387901306152 sec.
Downloaded file (189.0 KiB) in 1.1095590591430664 sec.


In [28]:
study_sets_by_study = dict()
for study in studies:
    study_sets_by_study[study['name']] = study['study_set']

In [15]:
pd.DataFrame(data=studies, columns=['study_set', 'name', 'directory'])

,study_set,name,directory
0,mearec_neuronexus,mearec_neuronexus_noise10_K10_C32,kbucket://15734439d8cf/groundtruth/mearec_synt...
1,mearec_neuronexus,mearec_neuronexus_noise10_K20_C32,kbucket://15734439d8cf/groundtruth/mearec_synt...
2,mearec_neuronexus,mearec_neuronexus_noise10_K40_C32,kbucket://15734439d8cf/groundtruth/mearec_synt...
3,mearec_neuronexus,mearec_neuronexus_noise20_K10_C32,kbucket://15734439d8cf/groundtruth/mearec_synt...
4,mearec_neuronexus,mearec_neuronexus_noise20_K20_C32,kbucket://15734439d8cf/groundtruth/mearec_synt...
5,mearec_neuronexus,mearec_neuronexus_noise20_K40_C32,kbucket://15734439d8cf/groundtruth/mearec_synt...
6,visapy_mea,visapy_mea,kbucket://15734439d8cf/groundtruth/visapy_mea
7,magland_synth,magland_synth_noise10_K10_C4,kbucket://15734439d8cf/groundtruth/magland_syn...
8,magland_synth,magland_synth_noise10_K10_C8,kbucket://15734439d8cf/groundtruth/magland_syn...
9,magland_synth,magland_synth_noise10_K20_C4,kbucket://15734439d8cf/groundtruth/magland_syn...


In [31]:
study_set='visapy_mea'
recordings2 = [dict(study=rec['study'], name=rec['name'], **rec['summary']['computed_info']) for rec in recordings]
recordings2 = [rec for rec in recordings2 if study_sets_by_study[rec['study']]==study_set]
pd.DataFrame(data=recordings2, columns=['study', 'name', 'samplerate', 'num_channels', 'duration_sec'])

,study,name,samplerate,num_channels,duration_sec
0,visapy_mea,set1,32000.0,30,300.0
1,visapy_mea,set2,32000.0,30,300.0
2,visapy_mea,set3,32000.0,30,300.0
3,visapy_mea,set4,32000.0,30,300.0
4,visapy_mea,set5,32000.0,30,300.0
5,visapy_mea,set6,32000.0,30,300.0


In [38]:
sorting_results[0]['sorter']['code']

'ms4'

In [40]:
sorter_codes

['yass', 'irc', 'ms4', 'sc']

In [63]:
all_elapsed = [SR['execution_stats']['elapsed_sec'] for SR in sorting_results]
total_elapsed_sec = np.sum(all_elapsed)
total_elapsed_hr = total_elapsed_sec/60/60
print('Total elapsed: {:.2f} hours -- {} sortings'.format(total_elapsed_hr, len(all_elapsed)))

Total elapsed: 103.18 hours -- 1552 sortings


In [62]:
sorter_codes=list(set([SR['sorter']['code'] for SR in sorting_results]))
for code in sorter_codes:
    all_elapsed = [SR['execution_stats']['elapsed_sec'] for SR in sorting_results if SR['sorter']['code']==code]
    total_elapsed_sec = np.sum(all_elapsed)
    total_elapsed_hr = total_elapsed_sec/60/60
    print('Total elapsed for {}: {:.2f} hours -- {} recordings'.format(code, total_elapsed_hr, len(all_elapsed)))

Total elapsed for yass: 16.69 hours -- 388 recordings
Total elapsed for irc: 14.50 hours -- 388 recordings
Total elapsed for ms4: 26.21 hours -- 388 recordings
Total elapsed for sc: 45.78 hours -- 388 recordings


In [61]:
study_sets = list(set(study_sets_by_study.values()))
for study_set in study_sets:
    print('-----------------------------------------')
    print('STUDY SET: {}'.format(study_set))
    sorting_results2 = [SR for SR in sorting_results if study_sets_by_study[SR['recording']['study']]==study_set]
    for code in sorter_codes:
        all_elapsed = [SR['execution_stats']['elapsed_sec'] for SR in sorting_results2 if SR['sorter']['code']==code]
        total_elapsed_sec = np.sum(all_elapsed)
        total_elapsed_hr = total_elapsed_sec/60/60
        print('Total elapsed for {}: {:.2f} hours -- {:.2f} minutes per recording -- {} recordings'.format(code, total_elapsed_hr, total_elapsed_hr/len(all_elapsed)*60, len(all_elapsed)))

-----------------------------------------
STUDY SET: magland_synth
Total elapsed for yass: 0.68 hours -- 0.51 minutes per recording -- 80 recordings
Total elapsed for irc: 1.48 hours -- 1.11 minutes per recording -- 80 recordings
Total elapsed for ms4: 1.53 hours -- 1.15 minutes per recording -- 80 recordings
Total elapsed for sc: 4.93 hours -- 3.70 minutes per recording -- 80 recordings
-----------------------------------------
STUDY SET: bionet
Total elapsed for yass: 6.21 hours -- 10.36 minutes per recording -- 36 recordings
Total elapsed for irc: 7.48 hours -- 12.47 minutes per recording -- 36 recordings
Total elapsed for ms4: 4.97 hours -- 8.28 minutes per recording -- 36 recordings
Total elapsed for sc: 12.01 hours -- 20.02 minutes per recording -- 36 recordings
-----------------------------------------
STUDY SET: paired
Total elapsed for yass: 4.73 hours -- 1.96 minutes per recording -- 145 recordings
Total elapsed for irc: 2.43 hours -- 1.01 minutes per recording -- 145 recordi